# Managing State in Multi-Agent Applications
In a multi-agent system, it's often useful to save the current state of agents, teams, and termination conditions. This allows us to load them back later, which is especially helpful in web applications where endpoints handle requests without storing persistent state.

In this guide, we'll explore how to save and load the state of agents and teams.

### Saving and Loading Agents
To save an agent’s state, we use the `save_state()` method. Let's create an assistant agent that shares fun facts about animals.

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an assistant agent
assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a fun fact expert about animals.",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-mini",
    ),
)

# Ask for a fun fact about dolphins
response = await assistant_agent.on_messages(
    [TextMessage(content="Tell me a fun fact about dolphins.", source="user")], CancellationToken()
)
print(response.chat_message.content)


In [ ]:

# Save the agent's state
agent_state = await assistant_agent.save_state()
print(agent_state)

In [ ]:

# Create a new agent instance and load the saved state
new_assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a fun fact expert about animals.",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-mini",
    ),
)
await new_assistant_agent.load_state(agent_state)

# Ask about the previous fun fact
response = await new_assistant_agent.on_messages(
    [TextMessage(content="What was the fun fact you shared about dolphins?", source="user")], CancellationToken()
)
print(response.chat_message.content)


### Saving and Loading Teams
We can also save and load the state of a team. When we save a team’s state, it includes the state of all agents within it.

##### Creating a Team
Let's create a team where an agent shares random animal facts.

In [ ]:
# Create an assistant agent
assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a fun fact expert about animals.",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-mini",
    ),
)

# Define a team that cycles through agents
agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))

# Ask for an animal fact
stream = agent_team.run_stream(task="Tell me a surprising fact about octopuses.")
await Console(stream)

# Save the team's state
team_state = await agent_team.save_state()

### Resetting the Team
If we reset the team and ask about the last shared fact, it won’t remember because the state was not loaded.

In [ ]:
# Reset the team
await agent_team.reset()

# Try asking again
stream = agent_team.run_stream(task="What was the last animal fact you shared?")
await Console(stream)

### Loading the Team's State
Now, let's load the saved state and ask the same question again.

In [ ]:
# Load the saved state
await agent_team.load_state(team_state)

# Ask again
stream = agent_team.run_stream(task="What was the last animal fact you shared?")
await Console(stream)


### Persisting State to a File
We can store the team’s state in a file (or a database) and reload it later.

In [ ]:
import json

# Save state to a file
with open("team_state.json", "w") as f:
    json.dump(team_state, f)

# Load state from a file
with open("team_state.json", "r") as f:
    team_state = json.load(f)

# Create a new team and load the saved state
new_agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))
await new_agent_team.load_state(team_state)

# Ask again
stream = new_agent_team.run_stream(task="What was the last animal fact you shared?")
await Console(stream)


## Summary
* Agents and teams can save their state to remember past interactions.
* Resetting a team loses previous context unless the state is reloaded.
* States can be saved to a file for long-term storage and later use.
* This approach is useful in applications where maintaining history across different sessions is important.